Decision Tree Classifier

In [1]:
from warnings import filterwarnings
filterwarnings ('ignore')

Read dataset

In [3]:
import pandas as pd
df=pd.read_csv( 'iris.csv')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [5]:
df['species'].value_counts()

species
setosa        50
versicolor    50
virginica     50
Name: count, dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [7]:
df.isna().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

Preprocessing

Separating x and y

In [8]:
x=df.drop(labels=['species'],axis=1)
y=df[['species']]

In [9]:
x.head()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [10]:
y.head()

,species
0,setosa
1,setosa
2,setosa
3,setosa
4,setosa


Label Encode The Target Features

In [13]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y['species']= le.fit_transform(y['species'])

In [14]:
y

,species
0,0
1,0
2,0
3,0
4,0
...,...
145,2
146,2
147,2
148,2


Perform pipeline on x Features

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [23]:
num_pipe=Pipeline(steps=[('imputer',SimpleImputer(strategy='mean')),
                         ('scaler',StandardScaler())])

In [24]:
x_pre = num_pipe.fit_transform(x)
x_pre[0:5]

array([[-0.90068117,  1.01900435, -1.34022653, -1.3154443 ],
       [-1.14301691, -0.13197948, -1.34022653, -1.3154443 ],
       [-1.38535265,  0.32841405, -1.39706395, -1.3154443 ],
       [-1.50652052,  0.09821729, -1.2833891 , -1.3154443 ],
       [-1.02184904,  1.24920112, -1.34022653, -1.3154443 ]])

In [25]:
cols = num_pipe.get_feature_names_out()
cols

array(['sepal_length', 'sepal_width', 'petal_length', 'petal_width'],
      dtype=object)

In [26]:
x_pre = pd.DataFrame(x_pre,columns=cols)
x_pre.head()

,sepal_length,sepal_width,petal_length,petal_width
0,-0.900681,1.019004,-1.340227,-1.315444
1,-1.143017,-0.131979,-1.340227,-1.315444
2,-1.385353,0.328414,-1.397064,-1.315444
3,-1.506521,0.098217,-1.283389,-1.315444
4,-1.021849,1.249201,-1.340227,-1.315444


Train Test Split

In [31]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest =train_test_split(x_pre,y,test_size=0.33,random_state=21)

Built Decision Trees

In [32]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(xtrain,ytrain)

DecisionTreeClassifier()

In [33]:
#Evaluating for training
dtc.score(xtrain,ytrain)

1.0

In [34]:
#Evaluate for testing
dtc.score(xtest,ytest)

0.94

### Tune the Hyperparameter
1. max_deapth
2. min_sample_split
3. min_sample_leaf
4. criterian(gini,entropy)

In [55]:
params = {'max_depth':[2,3,4,5,6],
          'min_samples-split':[4,5,6,7,8],
          'criterion':['gini','entropy']}

In [57]:
from sklearn.model_selection import GridSearchCV
model = DecisionTreeClassifier(random_state=42)
gscv = GridSearchCV(model,param_grid=params,cv=5,scoring='f1_macro')
gscv.fit(xtrain,ytrain)

ValueError: Invalid parameter 'min_samples-split' for estimator DecisionTreeClassifier(max_depth=2, random_state=42). Valid parameters are: ['ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'random_state', 'splitter'].

In [45]:
gscv.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [46]:
gscv.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

In [49]:
best.dtc=gscv.best_estimator_
best.dtc

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [50]:
best_dtc.score(xtrain,ytrain)

NameError: name 'best_dtc' is not defined

In [51]:
best_dtc.score(xtest,ytest)

NameError: name 'best_dtc' is not defined

Visualize how the tree is built

In [59]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
plt.figure(figsize=(15.15))
plot_tree(best_dtc,filled=True,feature_names=(xtrain.columns))
plt.show()

TypeError: Value after * must be an iterable, not float

In [60]:
ytrain.value_counts()

species
2          35
1          33
0          32
Name: count, dtype: int64

Evaluate Best Model On Test Data

In [62]:
#predict values on train and test
ypred_tr = best_dtc.predict(xtrain)
ypred_ts = best_dtc.predict(xtest)


NameError: name 'best_dtc' is not defined

In [65]:
#confusion matrics
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
cf = ConfusionMatrixDisplay(ytest,ypred_ts)
cdf=ConfusionMatrixDisplay(cf,display_labels=best_dtc.classes_)
cdf.plot()

NameError: name 'ypred_ts' is not defined

In [64]:
ypred_ts

NameError: name 'ypred_ts' is not defined

Classification report

In [66]:
from sklearn.metrics import classification_report
print(classification_report(ytest,ypred_ts))

NameError: name 'ypred_ts' is not defined

Getting original labels

In [67]:
ypred_tr_lb =le.inverse_transform(ypred_tr)
ypred_ts_lb = le.inverse_transform(ypred_ts)
                            

NameError: name 'ypred_tr' is not defined

In [68]:
ypred_tr_lb

NameError: name 'ypred_tr_lb' is not defined

In [69]:
ypred_ts_lb

NameError: name 'ypred_ts_lb' is not defined